### Import des libraires :

In [1]:
pip install sentencepiece transformers scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.1 MB/s 
     |████████████████████████████████| 5.8 MB 51.5 MB/s 
     |████████████████████████████████| 89 kB 9.9 MB/s 
     |████████████████████████████████| 7.6 MB 58.6 MB/s 
     |████████████████████████████████| 182 kB 63.9 MB/s 


In [2]:
!pip install gensim==4.2.0 # -> pour relancer le modéle 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.7 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
import pickle
import pandas as pd
import numpy as np
import IPython.display
import re

# Permettra de créer des accés avec google drive
import os

# Pour l'optimisation des algos :
from sklearn.model_selection import GridSearchCV

# pour les algorithmes supervisés :
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier


# Pour le BOW :
from nltk.tokenize import word_tokenize

# Pour le tf-idf :
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import tensorflow_hub as tf_hub

# Pour word2vec :
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import gensim
from gensim.models import Word2Vec, word2vec

# Bert
import os
import transformers
from transformers import *

# Pour la visualisation des tokens :
from sklearn.feature_extraction.text import CountVectorizer
from yellowbrick.text import FreqDistVisualizer

# Pour les scores :
from sklearn import metrics
from sklearn.metrics import accuracy_score, jaccard_score, hamming_loss
from sklearn.metrics import roc_auc_score

# Pour supprimer les warnings :
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### import des datas :

In [6]:
df_modellisation = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/Stackoverflow/df_modellisation_part_1.pkl")

In [ ]:
df_modellisation

### Mise en place de la limite des 50 tags :

In [7]:
top_20_tags = pickle.load(open('/content/drive/My Drive/Colab Notebooks/Stackoverflow/first_20_tags.pkl', 'rb')) # pour le load

In [ ]:
top_20_tags

### Fonction pour les scores :

In [8]:
def dico_metric(dico, col) :

  """
  source : https://scikit-learn.org/stable/modules/model_evaluation.html
  Permet de créer un dictionnaire avec les scores des differents algorithmes.
  """
  dico[col]  = {'Accuracy': round(metrics.accuracy_score(y_test, y_pred), 2) ,
                'Hamming loss' : round(metrics.hamming_loss(y_test, y_pred), 2),
              #  'Jaccard_score_macro' : round(metrics.jaccard_score(y_test, y_pred, average="macro"), 2),
                'Jaccard_score_micro' : round(metrics.jaccard_score(y_test, y_pred, average="micro"), 2),
              #  'f1_macro_score' : round(metrics.f1_score(y_test, y_pred, average='macro'), 2),
                'f1_micro_score' : round(metrics.f1_score(y_test, y_pred, average='micro'), 2),
                'Recall_micro_score' : round(metrics.f1_score(y_test, y_pred, average='micro'), 2),
              #  'Recall_macro_score' : round(metrics.f1_score(y_test, y_pred, average='macro'), 2),
                }
  return dico

# __Algorithme utilisant le BOW comme X :__

### Utiliser le bow en pickle :

In [10]:
bow = pickle.load(open('/content/drive/My Drive/Colab Notebooks/Stackoverflow/bow.pkl', 'rb')) # pour le load

### Train test via BOW :

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix

In [10]:
y = df_modellisation['tags_final']
multi_lab = MultiLabelBinarizer(classes= top_20_tags)
Y = multi_lab.fit_transform(y)

print("Affichage des classes du multilabel :")
display(multi_lab.classes_)

Affichage des classes du multilabel :


array(['python', 'javascript', 'java', 'reactjs', 'excel', 'r', 'c#',
       'c++', 'android', 'pandas', 'html', 'vba', 'node.js', 'css',
       'flutter', 'typescript', 'dataframe', 'sql', 'php'], dtype=object)

#### pickle :

In [12]:
multi_lab = pickle.dump(multi_lab, open('/content/drive/My Drive/Colab Notebooks/Stackoverflow/multi_lab.pkl', 'wb')) # pour le save

In [14]:
X = bow # a modifier par tfidf et par wordtovec
y = Y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

NameError: ignored

### Création d'une liste pour les scores des métrics :

In [ ]:
model_metrics_bow = {} # Garde en mémoire les scores

## Regression logistique :

In [12]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain

In [38]:
from sklearn.naive_bayes import MultinomialNB
def regression_log (X_train, y_train, X_test) :
  
  model_log = OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None,))
  model_log.fit(X_train, y_train)
  y_pred = model_log.predict(X_test)
  
  return y_pred

In [11]:
from sklearn.linear_model import LogisticRegression
def regression_log (X_train, y_train, X_test) :
  
  model_log = OneVsRestClassifier( LogisticRegression(
      class_weight= 'balanced',random_state = 0))
  model_log.fit(X_train, y_train)
  y_pred = model_log.predict(X_test)
  
  return y_pred

In [ ]:
%%time
y_pred, model_log = regression_log (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric(model_metrics_bow, "reg_log")

CPU times: user 7.33 s, sys: 104 ms, total: 7.43 s
Wall time: 9.21 s


{'reg_log': {'Accuracy': 0.02,
  'Hamming loss': 0.27,
  'Jaccard_score_micro': 0.15,
  'f1_micro_score': 0.26,
  'Recall_micro_score': 0.26}}

#### pickles :

In [ ]:
model_log = pickle.dump(model_log, open('/content/drive/My Drive/Colab Notebooks/Stackoverflow/model_log_v2.pkl', 'wb')) # pour le save

## Random Forest :

In [12]:
def random_forest (X_train, y_train, X_test) :

  model_rf = OneVsRestClassifier(RandomForestClassifier())
  model_rf.fit(X_train, y_train)
  y_pred = model_rf.predict(X_test) 
  
  return y_pred

In [ ]:
%%time
y_pred = random_forest (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_bow, "random_forest")

CPU times: user 6min 38s, sys: 1.78 s, total: 6min 40s
Wall time: 6min 41s


{'reg_log': {'Accuracy': 0.1,
  'Hamming loss': 0.09,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32},
 'random_forest': {'Accuracy': 0.1,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.13,
  'f1_micro_score': 0.22,
  'Recall_micro_score': 0.22}}

## Decision tree : 

In [13]:
def decision_tree (X_train, y_train, X_test) :

  model_tree = OneVsRestClassifier(DecisionTreeClassifier())
  model_tree.fit(X_train, y_train)
  y_pred = model_tree.predict(X_test) 
  
  return y_pred 

In [ ]:
%%time
y_pred = decision_tree (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_bow, "decision_tree")

CPU times: user 25.8 s, sys: 59.7 ms, total: 25.9 s
Wall time: 25.8 s


{'reg_log': {'Accuracy': 0.1,
  'Hamming loss': 0.09,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32},
 'random_forest': {'Accuracy': 0.1,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.13,
  'f1_micro_score': 0.22,
  'Recall_micro_score': 0.22},
 'decision_tree': {'Accuracy': 0.08,
  'Hamming loss': 0.11,
  'Jaccard_score_micro': 0.13,
  'f1_micro_score': 0.24,
  'Recall_micro_score': 0.24}}

## KNN : 

In [14]:
def KNeighbors (X_train, y_train, X_test) :

  model_knn = OneVsRestClassifier(KNeighborsClassifier())
  model_knn.fit(X_train, y_train)
  y_pred = model_knn.predict(X_test)
  
  return y_pred

In [ ]:
%%time
y_pred = KNeighbors (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_bow, "knn")

CPU times: user 2min 44s, sys: 1.95 s, total: 2min 46s
Wall time: 2min 46s


{'reg_log': {'Accuracy': 0.1,
  'Hamming loss': 0.09,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32},
 'random_forest': {'Accuracy': 0.1,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.13,
  'f1_micro_score': 0.22,
  'Recall_micro_score': 0.22},
 'decision_tree': {'Accuracy': 0.08,
  'Hamming loss': 0.11,
  'Jaccard_score_micro': 0.13,
  'f1_micro_score': 0.24,
  'Recall_micro_score': 0.24},
 'knn': {'Accuracy': 0.08,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.09,
  'f1_micro_score': 0.17,
  'Recall_micro_score': 0.17}}

## Xgboost :

In [15]:
def xgboost (X_train, y_train, X_test) :

  model_xgb = OneVsRestClassifier(XGBClassifier())
  model_xgb.fit(X_train, y_train)
  y_pred = model_xgb.predict(X_test)  
  return y_pred

In [ ]:
%%time
y_pred = xgboost (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_bow, "xgboost")

CPU times: user 27.3 s, sys: 145 ms, total: 27.5 s
Wall time: 27.3 s


{'reg_log': {'Accuracy': 0.1,
  'Hamming loss': 0.09,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32},
 'random_forest': {'Accuracy': 0.1,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.13,
  'f1_micro_score': 0.22,
  'Recall_micro_score': 0.22},
 'decision_tree': {'Accuracy': 0.08,
  'Hamming loss': 0.11,
  'Jaccard_score_micro': 0.13,
  'f1_micro_score': 0.24,
  'Recall_micro_score': 0.24},
 'knn': {'Accuracy': 0.08,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.09,
  'f1_micro_score': 0.17,
  'Recall_micro_score': 0.17},
 'xgboost': {'Accuracy': 0.07,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.1,
  'f1_micro_score': 0.18,
  'Recall_micro_score': 0.18}}

#### pickle du modéle :

In [ ]:
model_xgb = pickle.dump(model_xgb, open('./model_xgb.pkl', 'wb')) # pour le save

## Gradient Boosting :

In [16]:
def gradient_boost (X_train, y_train, X_test,) :
  
  model_svm = OneVsRestClassifier(GradientBoostingClassifier())
  model_svm.fit(X_train, y_train)
  y_pred = model_svm.predict(X_test)
  
  return y_pred

In [ ]:
%%time
y_pred = gradient_boost (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_bow, "gradient_boost")

CPU times: user 44.4 s, sys: 345 ms, total: 44.8 s
Wall time: 44.7 s


{'reg_log': {'Accuracy': 0.1,
  'Hamming loss': 0.09,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32},
 'random_forest': {'Accuracy': 0.1,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.13,
  'f1_micro_score': 0.22,
  'Recall_micro_score': 0.22},
 'decision_tree': {'Accuracy': 0.08,
  'Hamming loss': 0.11,
  'Jaccard_score_micro': 0.13,
  'f1_micro_score': 0.24,
  'Recall_micro_score': 0.24},
 'knn': {'Accuracy': 0.08,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.09,
  'f1_micro_score': 0.17,
  'Recall_micro_score': 0.17},
 'xgboost': {'Accuracy': 0.07,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.1,
  'f1_micro_score': 0.18,
  'Recall_micro_score': 0.18},
 'gradient_boost': {'Accuracy': 0.08,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.2,
  'Recall_micro_score': 0.2}}

## AdaBoostClassifier :

In [17]:
from sklearn.ensemble import AdaBoostClassifier

In [18]:
def Ada_Boost_Classifier (X_train, y_train, X_test) :
  
  model_adab = OneVsRestClassifier(AdaBoostClassifier())
  model_adab.fit(X_train, y_train)
  y_pred = model_adab.predict(X_test)
  
  return y_pred

In [ ]:
%%time
y_pred = Ada_Boost_Classifier (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_bow, "Ada_Boost_Classifier")

CPU times: user 21.8 s, sys: 262 ms, total: 22.1 s
Wall time: 22.1 s


{'reg_log': {'Accuracy': 0.1,
  'Hamming loss': 0.09,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32},
 'random_forest': {'Accuracy': 0.1,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.13,
  'f1_micro_score': 0.22,
  'Recall_micro_score': 0.22},
 'decision_tree': {'Accuracy': 0.08,
  'Hamming loss': 0.11,
  'Jaccard_score_micro': 0.13,
  'f1_micro_score': 0.24,
  'Recall_micro_score': 0.24},
 'knn': {'Accuracy': 0.08,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.09,
  'f1_micro_score': 0.17,
  'Recall_micro_score': 0.17},
 'xgboost': {'Accuracy': 0.07,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.1,
  'f1_micro_score': 0.18,
  'Recall_micro_score': 0.18},
 'gradient_boost': {'Accuracy': 0.08,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.2,
  'Recall_micro_score': 0.2},
 'Ada_Boost_Classifier': {'Accuracy': 0.09,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.13,
  'f1_micro_score': 0.23,
  'Reca

### Dictionnaire des scores :

In [ ]:
score_bow = pd.DataFrame.from_dict(model_metrics_bow)
# score_bow = score_bow.style.applymap(color_yellow).applymap(make_bold)
print("-"*83)
print("-"*35 + f"Via le BOW :" + "-"*36)
print("-"*83)
display(score_bow)

-----------------------------------------------------------------------------------
-----------------------------------Via le BOW :------------------------------------
-----------------------------------------------------------------------------------


,reg_log,random_forest,decision_tree,knn,xgboost,gradient_boost,Ada_Boost_Classifier
Accuracy,0.10,0.10,0.08,0.08,0.07,0.08,0.09
Hamming loss,0.09,0.07,0.11,0.07,0.06,0.06,0.06
Jaccard_score_micro,0.19,0.13,0.13,0.09,0.10,0.11,0.13
f1_micro_score,0.32,0.22,0.24,0.17,0.18,0.20,0.23
Recall_micro_score,0.32,0.22,0.24,0.17,0.18,0.20,0.23


#### Sauvegarde du résultat :

In [ ]:
score_bow.to_csv("/content/drive/My Drive/Colab Notebooks/Stackoverflow/Resultat_score_bow.csv", index=True) 
# Cela permettra d'éviter d'attendre de relancer le notebook.

# __Algorithme utilisant le tf-idf comme X :__

Ayant déjà crée un TF-IDF sur le notebook précédent, on va donc réutiliser notre TF-IDF pour l'utiliser pour notre algorithme.

### Utiliser le tf-idf en pickle :

In [ ]:
tf = pickle.load(open('/content/drive/My Drive/Colab Notebooks/Stackoverflow/tf.pkl', 'rb')) # pour le load"""

In [ ]:
idf = pickle.load(open('/content/drive/My Drive/Colab Notebooks/Stackoverflow/idf.pkl', 'rb')) # pour le load"""

### train test via Tf-IDF :

In [ ]:
X = idf # a modifier par tfidf et par wordtovec
y = Y
X_train, X_test, y_train1, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Création d'une liste pour les scores des métrics :

In [ ]:
model_metrics_idf = {} # Garde en mémoire les scores

## Regression logistique :

In [ ]:
%%time
y_pred,model = regression_log (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric(model_metrics_idf, "reg_log_")

CPU times: user 316 ms, sys: 969 µs, total: 317 ms
Wall time: 321 ms


{'reg_log_': {'Accuracy': 0.04,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.05,
  'f1_micro_score': 0.09,
  'Recall_micro_score': 0.09}}

## Random Forest :

In [ ]:
%%time
y_pred = random_forest (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_idf, "random_forest")

CPU times: user 7min 22s, sys: 2.39 s, total: 7min 24s
Wall time: 7min 22s


{'reg_log_': {'Accuracy': 0.04,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.05,
  'f1_micro_score': 0.09,
  'Recall_micro_score': 0.09},
 'random_forest': {'Accuracy': 0.09,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.2,
  'Recall_micro_score': 0.2}}

## Decision tree : 

In [ ]:
%%time
y_pred = decision_tree (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_idf, "decision_tree")

CPU times: user 38.6 s, sys: 38.9 ms, total: 38.6 s
Wall time: 38.4 s


{'reg_log_': {'Accuracy': 0.04,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.05,
  'f1_micro_score': 0.09,
  'Recall_micro_score': 0.09},
 'random_forest': {'Accuracy': 0.09,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.2,
  'Recall_micro_score': 0.2},
 'decision_tree': {'Accuracy': 0.09,
  'Hamming loss': 0.1,
  'Jaccard_score_micro': 0.14,
  'f1_micro_score': 0.24,
  'Recall_micro_score': 0.24}}

## KNN : 

In [ ]:
%%time
y_pred = KNeighbors (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_idf, "knn")

CPU times: user 2min 30s, sys: 931 ms, total: 2min 31s
Wall time: 2min 30s


{'reg_log_': {'Accuracy': 0.04,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.05,
  'f1_micro_score': 0.09,
  'Recall_micro_score': 0.09},
 'random_forest': {'Accuracy': 0.09,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.2,
  'Recall_micro_score': 0.2},
 'decision_tree': {'Accuracy': 0.09,
  'Hamming loss': 0.1,
  'Jaccard_score_micro': 0.14,
  'f1_micro_score': 0.24,
  'Recall_micro_score': 0.24},
 'knn': {'Accuracy': 0.1,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.2,
  'Recall_micro_score': 0.2}}

## Xgboost :

In [ ]:
%%time
y_pred = xgboost (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_idf, "xgboost")

CPU times: user 55.4 s, sys: 178 ms, total: 55.6 s
Wall time: 55.5 s


{'reg_log_': {'Accuracy': 0.04,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.05,
  'f1_micro_score': 0.09,
  'Recall_micro_score': 0.09},
 'random_forest': {'Accuracy': 0.09,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.2,
  'Recall_micro_score': 0.2},
 'decision_tree': {'Accuracy': 0.09,
  'Hamming loss': 0.1,
  'Jaccard_score_micro': 0.14,
  'f1_micro_score': 0.24,
  'Recall_micro_score': 0.24},
 'knn': {'Accuracy': 0.1,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.2,
  'Recall_micro_score': 0.2},
 'xgboost': {'Accuracy': 0.07,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.1,
  'f1_micro_score': 0.18,
  'Recall_micro_score': 0.18}}

## Gradient Boosting :

In [ ]:
%%time
y_pred = gradient_boost (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_idf, "gradient_boost")

CPU times: user 2min 28s, sys: 365 ms, total: 2min 29s
Wall time: 2min 28s


{'reg_log_': {'Accuracy': 0.04,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.05,
  'f1_micro_score': 0.09,
  'Recall_micro_score': 0.09},
 'random_forest': {'Accuracy': 0.09,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.2,
  'Recall_micro_score': 0.2},
 'decision_tree': {'Accuracy': 0.09,
  'Hamming loss': 0.1,
  'Jaccard_score_micro': 0.14,
  'f1_micro_score': 0.24,
  'Recall_micro_score': 0.24},
 'knn': {'Accuracy': 0.1,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.2,
  'Recall_micro_score': 0.2},
 'xgboost': {'Accuracy': 0.07,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.1,
  'f1_micro_score': 0.18,
  'Recall_micro_score': 0.18},
 'gradient_boost': {'Accuracy': 0.08,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.19,
  'Recall_micro_score': 0.19}}

## AdaBoostClassifier :

In [ ]:
%%time
y_pred = Ada_Boost_Classifier (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_idf, "Ada_Boost_Classifier")

CPU times: user 41.5 s, sys: 246 ms, total: 41.8 s
Wall time: 41.7 s


{'reg_log_': {'Accuracy': 0.04,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.05,
  'f1_micro_score': 0.09,
  'Recall_micro_score': 0.09},
 'random_forest': {'Accuracy': 0.09,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.2,
  'Recall_micro_score': 0.2},
 'decision_tree': {'Accuracy': 0.09,
  'Hamming loss': 0.1,
  'Jaccard_score_micro': 0.14,
  'f1_micro_score': 0.24,
  'Recall_micro_score': 0.24},
 'knn': {'Accuracy': 0.1,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.2,
  'Recall_micro_score': 0.2},
 'xgboost': {'Accuracy': 0.07,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.1,
  'f1_micro_score': 0.18,
  'Recall_micro_score': 0.18},
 'gradient_boost': {'Accuracy': 0.08,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.11,
  'f1_micro_score': 0.19,
  'Recall_micro_score': 0.19},
 'Ada_Boost_Classifier': {'Accuracy': 0.1,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.13,
  'f1_micro_score': 0.23,
  'Recall

### Dictionnaire des scores :

In [ ]:
score_tfidf = pd.DataFrame.from_dict(model_metrics_idf)

print("-"*83)
print("-"*34 + f"Via le TFIDF :" + "-"*35)
print("-"*83)
display(score_tfidf)

-----------------------------------------------------------------------------------
----------------------------------Via le TFIDF :-----------------------------------
-----------------------------------------------------------------------------------


,reg_log_,random_forest,decision_tree,knn,xgboost,gradient_boost,Ada_Boost_Classifier
Accuracy,0.04,0.09,0.09,0.10,0.07,0.08,0.10
Hamming loss,0.07,0.07,0.10,0.07,0.06,0.06,0.06
Jaccard_score_micro,0.05,0.11,0.14,0.11,0.10,0.11,0.13
f1_micro_score,0.09,0.20,0.24,0.20,0.18,0.19,0.23
Recall_micro_score,0.09,0.20,0.24,0.20,0.18,0.19,0.23


#### Sauvegarde du résultat :

In [ ]:
score_tfidf.to_csv("/content/drive/My Drive/Colab Notebooks/Stackoverflow/resultat_score_tfidf.csv", index=True)

# __Algorithme utilisant le word2vec :__

In [ ]:
import nltk
from nltk.corpus import stopwords
import string  # permet d'avoir accés à toute les ponctuations.
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def tokenize(text):

    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)

    try:
        res = word_tokenize(text, language='english')
    except TypeError:
        return text

    res = [token for token in res if token not in punctuation]
    res = [token for token in res if token not in stop_words]
   # res = [token for token in res if token not in letters]
    return res

### Connaître le nombre de mots max :

In [ ]:
# Pour connaitre le max len :
df_modellisation['length_corpus'] = df_modellisation['corpus_finish'].apply(lambda x : len(tokenize(x)))
print(f"Dans le corpus, le nombre de mots maximum est de : ", df_modellisation['length_corpus'].max())

Dans le corpus, le nombre de mots maximum est de :  334


### Train test via word2vec :

In [ ]:
corpus = df_modellisation['corpus_finish']
corpus_count =  334

### Mise en place de Word2vec :

In [ ]:
w2v_size=300
w2v_window=5
w2v_min_count=25
w2v_epochs=100
maxlen = 334 # taille des phrases à vérifier pour le max
sentences = df_modellisation['corpus_finish'].tolist()
sentences = [gensim.utils.simple_preprocess(text) for text in sentences]

### Entrainement du Word2vec :

In [ ]:
%%time
# Création et entraînement du modèle Word2Vec

print("Build & train Word2Vec model ...")
w2v_model = gensim.models.Word2Vec(min_count=w2v_min_count, window=w2v_window,
                                                vector_size=w2v_size,
                                                seed=42,
                                                workers=1)
#                                                workers=multiprocessing.cpu_count())
w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_epochs)
model_vectors = w2v_model.wv
w2v_words = model_vectors.index_to_key
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

Build & train Word2Vec model ...
Vocabulary size: 2168
Word2Vec trained
CPU times: user 2min 50s, sys: 1.11 s, total: 2min 51s
Wall time: 2min 57s


#### pickle du modéle :

In [ ]:
pickle.dump(w2v_model, open('/content/drive/My Drive/Colab Notebooks/Stackoverflow/w2v_model.pkl', 'wb'))
# w2v_model = pickle.load(open('/content/drive/My Drive/Colab Notebooks/Stackoverflow/w2v_model.pkl', 'rb')) # pour le load

### Tokennisation du modéle :

In [ ]:
%%time
# Préparation des sentences (tokenization)

print("Fit Tokenizer ...")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
x_sentences = pad_sequences(tokenizer.texts_to_sequences(sentences),
                                                     maxlen=maxlen,
                                                     padding='post') 
                                                   
num_words = len(tokenizer.word_index) + 1
print("Number of unique words: %i" % num_words)

Fit Tokenizer ...
Number of unique words: 30002
CPU times: user 2.24 s, sys: 109 ms, total: 2.35 s
Wall time: 2.63 s


### Création de la matrice d'embedding :

In [ ]:
# Création de la matrice d'embedding

print("Create Embedding matrix ...")
w2v_size = 300
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, w2v_size))
i=0
j=0
    
for word, idx in word_index.items():
    i +=1
    if word in w2v_words:
        j +=1
        embedding_vector = model_vectors[word]
        if embedding_vector is not None:
            embedding_matrix[idx] = model_vectors[word]
            
word_rate = np.round(j/i,4)
print("Word embedding rate : ", word_rate)
print("Embedding matrix: %s" % str(embedding_matrix.shape))

Create Embedding matrix ...
Word embedding rate :  0.0723
Embedding matrix: (30002, 300)


### Création du modéle :

In [ ]:
# Création du modèle

input=Input(shape=(len(x_sentences),maxlen),dtype='float64')
word_input=Input(shape=(maxlen,),dtype='float64')  
word_embedding=Embedding(input_dim=vocab_size,
                         output_dim=w2v_size,
                         weights = [embedding_matrix],
                         input_length=maxlen)(word_input)
word_vec=GlobalAveragePooling1D()(word_embedding)  
embed_model = Model([word_input],word_vec)

embed_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 334)]             0         
                                                                 
 embedding (Embedding)       (None, 334, 300)          9000600   
                                                                 
 global_average_pooling1d (G  (None, 300)              0         
 lobalAveragePooling1D)                                          
                                                                 
Total params: 9,000,600
Trainable params: 9,000,600
Non-trainable params: 0
_________________________________________________________________


#### Pickle de  x_sentences :

In [ ]:
pickle.dump(x_sentences, open('/content/drive/My Drive/Colab Notebooks/Stackoverflow/x_sentences.pkl', 'wb'))
# x_sentences = pickle.load(open('x_sentences.pkl', 'rb')) # pour le load

### Train test via Word2vec :

In [ ]:
X = x_sentences # a modifier par tfidf et par wordtovec
y = Y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Création d'une liste pour les scores des métrics :

In [ ]:
model_metrics_word2vec = {} # Garde en mémoire les scores

## Regression Logistique :

In [ ]:
%%time
y_pred, model_reg = regression_log (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "reg_log")

CPU times: user 5.58 s, sys: 307 ms, total: 5.89 s
Wall time: 5.45 s


{'reg_log': {'Accuracy': 0.0,
  'Hamming loss': 0.48,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.12,
  'Recall_micro_score': 0.12}}

## Random Forest :

In [ ]:
%%time
y_pred = random_forest (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "random_forest")

CPU times: user 6min 21s, sys: 8.58 s, total: 6min 30s
Wall time: 6min 32s


{'reg_log': {'Accuracy': 0.0,
  'Hamming loss': 0.48,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.12,
  'Recall_micro_score': 0.12},
 'random_forest': {'Accuracy': 0.0,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.0,
  'f1_micro_score': 0.01,
  'Recall_micro_score': 0.01}}

## Decision tree : 

In [ ]:
%%time
y_pred = decision_tree (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "decision_tree")

CPU times: user 1min 27s, sys: 150 ms, total: 1min 27s
Wall time: 1min 27s


{'reg_log': {'Accuracy': 0.0,
  'Hamming loss': 0.48,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.12,
  'Recall_micro_score': 0.12},
 'random_forest': {'Accuracy': 0.0,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.0,
  'f1_micro_score': 0.01,
  'Recall_micro_score': 0.01},
 'decision_tree': {'Accuracy': 0.04,
  'Hamming loss': 0.13,
  'Jaccard_score_micro': 0.09,
  'f1_micro_score': 0.16,
  'Recall_micro_score': 0.16}}

## KNN : 

In [ ]:
%%time
y_pred = KNeighbors (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "knn")

CPU times: user 5min 49s, sys: 11.4 s, total: 6min
Wall time: 3min 58s


{'reg_log': {'Accuracy': 0.0,
  'Hamming loss': 0.48,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.12,
  'Recall_micro_score': 0.12},
 'random_forest': {'Accuracy': 0.0,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.0,
  'f1_micro_score': 0.01,
  'Recall_micro_score': 0.01},
 'decision_tree': {'Accuracy': 0.04,
  'Hamming loss': 0.13,
  'Jaccard_score_micro': 0.09,
  'f1_micro_score': 0.16,
  'Recall_micro_score': 0.16},
 'knn': {'Accuracy': 0.03,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.03,
  'f1_micro_score': 0.05,
  'Recall_micro_score': 0.05}}

## Xgboost :

In [ ]:
%%time
y_pred = xgboost (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "xgboost")

CPU times: user 7min 27s, sys: 2.09 s, total: 7min 29s
Wall time: 7min 27s


{'reg_log': {'Accuracy': 0.0,
  'Hamming loss': 0.48,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.12,
  'Recall_micro_score': 0.12},
 'random_forest': {'Accuracy': 0.0,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.0,
  'f1_micro_score': 0.01,
  'Recall_micro_score': 0.01},
 'decision_tree': {'Accuracy': 0.04,
  'Hamming loss': 0.13,
  'Jaccard_score_micro': 0.09,
  'f1_micro_score': 0.16,
  'Recall_micro_score': 0.16},
 'knn': {'Accuracy': 0.03,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.03,
  'f1_micro_score': 0.05,
  'Recall_micro_score': 0.05},
 'xgboost': {'Accuracy': 0.02,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.03,
  'f1_micro_score': 0.05,
  'Recall_micro_score': 0.05}}

## Gradient Boosting :

In [ ]:
%%time
y_pred = gradient_boost (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "gradient_boost")

CPU times: user 22min 19s, sys: 2.22 s, total: 22min 21s
Wall time: 22min 24s


{'reg_log': {'Accuracy': 0.0,
  'Hamming loss': 0.48,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.12,
  'Recall_micro_score': 0.12},
 'random_forest': {'Accuracy': 0.0,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.0,
  'f1_micro_score': 0.01,
  'Recall_micro_score': 0.01},
 'decision_tree': {'Accuracy': 0.04,
  'Hamming loss': 0.13,
  'Jaccard_score_micro': 0.09,
  'f1_micro_score': 0.16,
  'Recall_micro_score': 0.16},
 'knn': {'Accuracy': 0.03,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.03,
  'f1_micro_score': 0.05,
  'Recall_micro_score': 0.05},
 'xgboost': {'Accuracy': 0.02,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.03,
  'f1_micro_score': 0.05,
  'Recall_micro_score': 0.05},
 'gradient_boost': {'Accuracy': 0.02,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.03,
  'f1_micro_score': 0.06,
  'Recall_micro_score': 0.06}}

## AdaBoostClassifier :

In [ ]:
%%time
y_pred = Ada_Boost_Classifier (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "Ada_Boost_Classifier")

CPU times: user 4min 23s, sys: 1.12 s, total: 4min 24s
Wall time: 4min 22s


{'reg_log': {'Accuracy': 0.0,
  'Hamming loss': 0.48,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.12,
  'Recall_micro_score': 0.12},
 'random_forest': {'Accuracy': 0.0,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.0,
  'f1_micro_score': 0.01,
  'Recall_micro_score': 0.01},
 'decision_tree': {'Accuracy': 0.04,
  'Hamming loss': 0.13,
  'Jaccard_score_micro': 0.09,
  'f1_micro_score': 0.16,
  'Recall_micro_score': 0.16},
 'knn': {'Accuracy': 0.03,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.03,
  'f1_micro_score': 0.05,
  'Recall_micro_score': 0.05},
 'xgboost': {'Accuracy': 0.02,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.03,
  'f1_micro_score': 0.05,
  'Recall_micro_score': 0.05},
 'gradient_boost': {'Accuracy': 0.02,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.03,
  'f1_micro_score': 0.06,
  'Recall_micro_score': 0.06},
 'Ada_Boost_Classifier': {'Accuracy': 0.0,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.0,
  'f1_micro_score': 0.0,
  'Recal

### Dictionnaire des scores :

In [ ]:
score_word2vec = pd.DataFrame.from_dict(model_metrics_word2vec)

print("-"*83)
print("-"*32 + f"Via le word2vec :" + "-"*34)
print("-"*83)
display(score_word2vec)

-----------------------------------------------------------------------------------
--------------------------------Via le word2vec :----------------------------------
-----------------------------------------------------------------------------------


,reg_log,random_forest,decision_tree,knn,xgboost,gradient_boost,Ada_Boost_Classifier
Accuracy,0.00,0.00,0.04,0.03,0.02,0.02,0.00
Hamming loss,0.48,0.07,0.13,0.07,0.07,0.07,0.07
Jaccard_score_micro,0.07,0.00,0.09,0.03,0.03,0.03,0.00
f1_micro_score,0.12,0.01,0.16,0.05,0.05,0.06,0.00
Recall_micro_score,0.12,0.01,0.16,0.05,0.05,0.06,0.00


#### Sauvegarde du résultat :

In [ ]:
score_word2vec.to_csv("/content/drive/My Drive/Colab Notebooks/Stackoverflow/resultat_score_word2vec.csv") # Cela permettra d'éviter d'attendre de relancer le notebook.

# __Algorithme utilisant le BERT :__

### Fonction pour le BERT :

In [19]:
# Fonction de préparation des sentences
def bert_inp_fct(sentences, bert_tokenizer, max_length) :
    input_ids=[]
    token_type_ids = []
    attention_mask=[]
    bert_inp_tot = []

    for sent in sentences:
        bert_inp = bert_tokenizer.encode_plus(sent,
                                              add_special_tokens = True,
                                              max_length = max_length,
                                              padding='max_length',
                                              return_attention_mask = True, 
                                              return_token_type_ids=True,
                                              truncation=True,
                                              return_tensors="tf")
    
        input_ids.append(bert_inp['input_ids'][0])
        token_type_ids.append(bert_inp['token_type_ids'][0])
        attention_mask.append(bert_inp['attention_mask'][0])
        bert_inp_tot.append((bert_inp['input_ids'][0], 
                             bert_inp['token_type_ids'][0], 
                             bert_inp['attention_mask'][0]))

    input_ids = np.asarray(input_ids)
    token_type_ids = np.asarray(token_type_ids)
    attention_mask = np.array(attention_mask)
    
    return input_ids, token_type_ids, attention_mask, bert_inp_tot


In [20]:
# Fonction de création des features
def feature_BERT_fct(model, model_type, sentences, max_length, b_size, mode='HF') :
    batch_size = b_size
    batch_size_pred = b_size
    bert_tokenizer = AutoTokenizer.from_pretrained(model_type)
    time1 = time.time()

    for step in range(len(sentences)//batch_size) :
        idx = step*batch_size
        input_ids, token_type_ids, attention_mask, bert_inp_tot = bert_inp_fct(sentences[idx:idx+batch_size], 
                                                                      bert_tokenizer, max_length)
        
        if mode=='HF' :    # Bert HuggingFace
            outputs = model.predict([input_ids, attention_mask, token_type_ids], batch_size=batch_size_pred)
            last_hidden_states = outputs.last_hidden_state

        if mode=='TFhub' : # Bert Tensorflow Hub
            text_preprocessed = {"input_word_ids" : input_ids, 
                                 "input_mask" : attention_mask, 
                                 "input_type_ids" : token_type_ids}
            outputs = model(text_preprocessed)
            last_hidden_states = outputs['sequence_output']
             
        if step ==0 :
            last_hidden_states_tot = last_hidden_states
            last_hidden_states_tot_0 = last_hidden_states
        else :
            last_hidden_states_tot = np.concatenate((last_hidden_states_tot,last_hidden_states))
    
    features_bert = np.array(last_hidden_states_tot).mean(axis=1)
    
    time2 = np.round(time.time() - time1,0)
    print("temps traitement : ", time2)
     
    return features_bert, last_hidden_states_tot

### Mise en place du BERT :

In [ ]:
"""pip install -U tensorflow-text==2.10.0"""

In [21]:
import tensorflow as tf
# import tensorflow_hub as hub
import tensorflow.keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

# Bert
import os
import transformers
from transformers import *

os.environ["TF_KERAS"]='1'

In [22]:
print(tf .__version__)
print(tensorflow.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.is_built_with_cuda())

2.9.2
2.9.2
Num GPUs Available:  1
True


In [23]:
max_length = 334
batch_size = 200
model_type = 'bert-base-uncased'
model = TFAutoModel.from_pretrained(model_type)
sentences = df_modellisation['corpus_finish'][:2000].tolist()

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

loading weights file tf_model.h5 from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tf_model.h5
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training

In [24]:
import time
# Création des features

features_bert, last_hidden_states_tot = feature_BERT_fct(model, model_type, sentences, 
                                                         max_length, batch_size, mode='HF')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_ac

1/1 [==============================] - 4s 4s/step
temps traitement :  74.0


#### pickle de features_bert :

In [ ]:
pickle.dump(features_bert, open("/content/drive/My Drive/Colab Notebooks/Stackoverflow/features_bert", 'wb'))

In [ ]:
features_bert = pickle.load(open("/content/drive/My Drive/Colab Notebooks/Stackoverflow/features_bert", 'rb')) # pour le load

### Train test via Bert :

In [26]:
X = features_bert
y = Y[:2000]
X_train, X_test, y_train,  y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Création d'une liste pour les scores des métrics :

In [28]:
model_metrics_bert = {} # Garde en mémoire les scores

## Regression Logistique :

In [29]:
%%time
y_pred = regression_log (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "reg_log")

CPU times: user 4.47 s, sys: 2.64 s, total: 7.11 s
Wall time: 8.19 s


{'reg_log': {'Accuracy': 0.06,
  'Hamming loss': 0.13,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32}}

## Random Forest :

In [30]:
%%time
y_pred = random_forest (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "random_forest")

CPU times: user 1min 14s, sys: 52 ms, total: 1min 14s
Wall time: 1min 19s


{'reg_log': {'Accuracy': 0.06,
  'Hamming loss': 0.13,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32},
 'random_forest': {'Accuracy': 0.0,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.0,
  'f1_micro_score': 0.01,
  'Recall_micro_score': 0.01}}

## Decision Tree Classifier :

In [31]:
%%time
y_pred = decision_tree (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "decision_tree")

CPU times: user 51.1 s, sys: 44.5 ms, total: 51.1 s
Wall time: 52.3 s


{'reg_log': {'Accuracy': 0.06,
  'Hamming loss': 0.13,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32},
 'random_forest': {'Accuracy': 0.0,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.0,
  'f1_micro_score': 0.01,
  'Recall_micro_score': 0.01},
 'decision_tree': {'Accuracy': 0.04,
  'Hamming loss': 0.12,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.14,
  'Recall_micro_score': 0.14}}

## KNN : 

In [32]:
%%time
y_pred = KNeighbors (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "knn")

CPU times: user 1.49 s, sys: 511 ms, total: 2 s
Wall time: 1.95 s


{'reg_log': {'Accuracy': 0.06,
  'Hamming loss': 0.13,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32},
 'random_forest': {'Accuracy': 0.0,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.0,
  'f1_micro_score': 0.01,
  'Recall_micro_score': 0.01},
 'decision_tree': {'Accuracy': 0.04,
  'Hamming loss': 0.12,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.14,
  'Recall_micro_score': 0.14},
 'knn': {'Accuracy': 0.05,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.13,
  'Recall_micro_score': 0.13}}

## Xgboost :

In [33]:
%%time
y_pred = xgboost (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "xgboost")

CPU times: user 2min 10s, sys: 365 ms, total: 2min 10s
Wall time: 2min 14s


{'reg_log': {'Accuracy': 0.06,
  'Hamming loss': 0.13,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32},
 'random_forest': {'Accuracy': 0.0,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.0,
  'f1_micro_score': 0.01,
  'Recall_micro_score': 0.01},
 'decision_tree': {'Accuracy': 0.04,
  'Hamming loss': 0.12,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.14,
  'Recall_micro_score': 0.14},
 'knn': {'Accuracy': 0.05,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.13,
  'Recall_micro_score': 0.13},
 'xgboost': {'Accuracy': 0.02,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.03,
  'f1_micro_score': 0.06,
  'Recall_micro_score': 0.06}}

## Gradient Boosting :

In [34]:
%%time
y_pred = gradient_boost (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "gradient_boost")

CPU times: user 11min 20s, sys: 714 ms, total: 11min 21s
Wall time: 11min 20s


{'reg_log': {'Accuracy': 0.06,
  'Hamming loss': 0.13,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32},
 'random_forest': {'Accuracy': 0.0,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.0,
  'f1_micro_score': 0.01,
  'Recall_micro_score': 0.01},
 'decision_tree': {'Accuracy': 0.04,
  'Hamming loss': 0.12,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.14,
  'Recall_micro_score': 0.14},
 'knn': {'Accuracy': 0.05,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.13,
  'Recall_micro_score': 0.13},
 'xgboost': {'Accuracy': 0.02,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.03,
  'f1_micro_score': 0.06,
  'Recall_micro_score': 0.06},
 'gradient_boost': {'Accuracy': 0.02,
  'Hamming loss': 0.08,
  'Jaccard_score_micro': 0.04,
  'f1_micro_score': 0.07,
  'Recall_micro_score': 0.07}}

## AdaBoostClassifier :

In [35]:
%%time
y_pred = Ada_Boost_Classifier (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "Ada_Boost_Classifier")

CPU times: user 2min 9s, sys: 122 ms, total: 2min 9s
Wall time: 2min 8s


{'reg_log': {'Accuracy': 0.06,
  'Hamming loss': 0.13,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32},
 'random_forest': {'Accuracy': 0.0,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.0,
  'f1_micro_score': 0.01,
  'Recall_micro_score': 0.01},
 'decision_tree': {'Accuracy': 0.04,
  'Hamming loss': 0.12,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.14,
  'Recall_micro_score': 0.14},
 'knn': {'Accuracy': 0.05,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.07,
  'f1_micro_score': 0.13,
  'Recall_micro_score': 0.13},
 'xgboost': {'Accuracy': 0.02,
  'Hamming loss': 0.07,
  'Jaccard_score_micro': 0.03,
  'f1_micro_score': 0.06,
  'Recall_micro_score': 0.06},
 'gradient_boost': {'Accuracy': 0.02,
  'Hamming loss': 0.08,
  'Jaccard_score_micro': 0.04,
  'f1_micro_score': 0.07,
  'Recall_micro_score': 0.07},
 'Ada_Boost_Classifier': {'Accuracy': 0.08,
  'Hamming loss': 0.08,
  'Jaccard_score_micro': 0.12,
  'f1_micro_score': 0.21,
  'R

### Dictionnaire des scores :

In [36]:
score_bert = pd.DataFrame.from_dict(model_metrics_bert)

print("-"*100)
print("-"*42 + f"Via le BERT :" + "-"*42)
print("-"*100)
display(score_bert)

----------------------------------------------------------------------------------------------------
------------------------------------------Via le BERT :------------------------------------------
----------------------------------------------------------------------------------------------------


,reg_log,random_forest,decision_tree,knn,xgboost,gradient_boost,Ada_Boost_Classifier
Accuracy,0.06,0.00,0.04,0.05,0.02,0.02,0.08
Hamming loss,0.13,0.07,0.12,0.07,0.07,0.08,0.08
Jaccard_score_micro,0.19,0.00,0.07,0.07,0.03,0.04,0.12
f1_micro_score,0.32,0.01,0.14,0.13,0.06,0.07,0.21
Recall_micro_score,0.32,0.01,0.14,0.13,0.06,0.07,0.21


#### Sauvegarde du résultat :

In [37]:
score_bert.to_csv("/content/drive/My Drive/Colab Notebooks/Stackoverflow/score_bert.csv") # Cela permettra d'éviter d'attendre de relancer le notebook.

# __Algorithme utilisant USE :__

### Mise en place du USE :

In [ ]:
import tensorflow as tf
# import tensorflow_hub as hub
import tensorflow.keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import tensorflow_hub as hub

In [ ]:
def regression_log (X_train, y_train, X_test) :
  
  model_log = OneVsRestClassifier(LinearSVC(), n_jobs=1)
  model_log.fit(X_train, y_train)
  y_pred = model_log.predict(X_test)
  
  return y_pred, model_log

In [ ]:
%%time
module_url = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

CPU times: user 12 s, sys: 3.77 s, total: 15.7 s
Wall time: 16.4 s


In [ ]:
def embed(input):
  return module_url(input)

#### Mise au format array pour le X :

In [ ]:
%%time
embedding_sentence = embed(df_modellisation['corpus_finish'].tolist())
embedding_sentence = np.array(embedding_sentence)

CPU times: user 17.6 s, sys: 1.83 s, total: 19.5 s
Wall time: 16.3 s


#### pickle de embedding_sentence :

In [ ]:
pickle.dump(embedding_sentence, open('/content/drive/My Drive/Colab Notebooks/Stackoverflow/embedding_sentence.pkl', 'wb'))
# embedding_sentence = pickle.load(open('embedding_sentence.pkl', 'rb')) # pour le load

### Train test via USE :

In [ ]:
X = embedding_sentence # pour la méthode USE
y = Y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Création d'une liste pour les scores des métrics :

In [ ]:
model_metrics_use = {}

## Regression logistique :

In [ ]:
%%time
y_pred,model_reg= regression_log (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "reg_log")

CPU times: user 44.2 s, sys: 1.37 s, total: 45.6 s
Wall time: 46.4 s


{'reg_log': {'Accuracy': 0.25,
  'Hamming loss': 0.05,
  'Jaccard_score_micro': 0.29,
  'f1_micro_score': 0.45,
  'Recall_micro_score': 0.45}}

#### Pickle du résultat :

In [ ]:
model_reg = pickle.dump(model_reg, open('/content/drive/My Drive/Colab Notebooks/Stackoverflow/model_log.pkl', 'wb')) # pour le save

## Random Forest :

In [ ]:
%%time
y_pred = random_forest (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "random_forest")

CPU times: user 51min 40s, sys: 25.3 s, total: 52min 5s
Wall time: 52min 6s


{'reg_log': {'Accuracy': 0.25,
  'Hamming loss': 0.05,
  'Jaccard_score_micro': 0.29,
  'f1_micro_score': 0.45,
  'Recall_micro_score': 0.45},
 'random_forest': {'Accuracy': 0.13,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.16,
  'f1_micro_score': 0.28,
  'Recall_micro_score': 0.28}}

## Decision tree : 

In [ ]:
%%time
y_pred = decision_tree (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "decision_tree")

CPU times: user 28min 23s, sys: 1.42 s, total: 28min 24s
Wall time: 28min 19s


{'reg_log': {'Accuracy': 0.25,
  'Hamming loss': 0.05,
  'Jaccard_score_micro': 0.29,
  'f1_micro_score': 0.45,
  'Recall_micro_score': 0.45},
 'random_forest': {'Accuracy': 0.13,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.16,
  'f1_micro_score': 0.28,
  'Recall_micro_score': 0.28},
 'decision_tree': {'Accuracy': 0.13,
  'Hamming loss': 0.1,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.31,
  'Recall_micro_score': 0.31}}

## KNN : 

In [ ]:
%%time
y_pred = KNeighbors (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "knn")

CPU times: user 8min 28s, sys: 1min 1s, total: 9min 30s
Wall time: 5min 42s


{'reg_log': {'Accuracy': 0.25,
  'Hamming loss': 0.05,
  'Jaccard_score_micro': 0.29,
  'f1_micro_score': 0.45,
  'Recall_micro_score': 0.45},
 'random_forest': {'Accuracy': 0.13,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.16,
  'f1_micro_score': 0.28,
  'Recall_micro_score': 0.28},
 'decision_tree': {'Accuracy': 0.13,
  'Hamming loss': 0.1,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.31,
  'Recall_micro_score': 0.31},
 'knn': {'Accuracy': 0.27,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.28,
  'f1_micro_score': 0.44,
  'Recall_micro_score': 0.44}}

## Xgboost :

In [ ]:
%%time
y_pred = xgboost (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "xgboost")

CPU times: user 36min 5s, sys: 7.65 s, total: 36min 12s
Wall time: 36min 3s


{'reg_log': {'Accuracy': 0.25,
  'Hamming loss': 0.05,
  'Jaccard_score_micro': 0.29,
  'f1_micro_score': 0.45,
  'Recall_micro_score': 0.45},
 'random_forest': {'Accuracy': 0.13,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.16,
  'f1_micro_score': 0.28,
  'Recall_micro_score': 0.28},
 'decision_tree': {'Accuracy': 0.13,
  'Hamming loss': 0.1,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.31,
  'Recall_micro_score': 0.31},
 'knn': {'Accuracy': 0.27,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.28,
  'f1_micro_score': 0.44,
  'Recall_micro_score': 0.44},
 'xgboost': {'Accuracy': 0.22,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.27,
  'f1_micro_score': 0.42,
  'Recall_micro_score': 0.42}}

## Gradient Boosting :

In [ ]:
"""%%time
y_pred = gradient_boost (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "gradient_boost")"""
# Durée 4h22

"""
CPU times: user 4h 23min 37s, sys: 17.3 s, total: 4h 23min 54s
Wall time: 4h 22min 52s
  'gradient_boost': {'Accuracy': 0.22,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.26,
  'f1_micro_score': 0.42,
  'Recall_micro_score': 0.42}}
"""
# Non pertinent

CPU times: user 4h 23min 37s, sys: 17.3 s, total: 4h 23min 54s
Wall time: 4h 22min 52s


{'reg_log': {'Accuracy': 0.25,
  'Hamming loss': 0.05,
  'Jaccard_score_micro': 0.29,
  'f1_micro_score': 0.45,
  'Recall_micro_score': 0.45},
 'random_forest': {'Accuracy': 0.14,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.17,
  'f1_micro_score': 0.29,
  'Recall_micro_score': 0.29},
 'decision_tree': {'Accuracy': 0.14,
  'Hamming loss': 0.1,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.32,
  'Recall_micro_score': 0.32},
 'knn': {'Accuracy': 0.27,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.28,
  'f1_micro_score': 0.44,
  'Recall_micro_score': 0.44},
 'xgboost': {'Accuracy': 0.22,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.27,
  'f1_micro_score': 0.42,
  'Recall_micro_score': 0.42},
 'gradient_boost': {'Accuracy': 0.22,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.26,
  'f1_micro_score': 0.42,
  'Recall_micro_score': 0.42}}

## AdaBoostClassifier :

In [ ]:
%%time
y_pred = Ada_Boost_Classifier (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "Ada_Boost_Classifier")

CPU times: user 49min 55s, sys: 3.04 s, total: 49min 58s
Wall time: 49min 52s


{'reg_log': {'Accuracy': 0.25,
  'Hamming loss': 0.05,
  'Jaccard_score_micro': 0.29,
  'f1_micro_score': 0.45,
  'Recall_micro_score': 0.45},
 'random_forest': {'Accuracy': 0.13,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.16,
  'f1_micro_score': 0.28,
  'Recall_micro_score': 0.28},
 'decision_tree': {'Accuracy': 0.13,
  'Hamming loss': 0.1,
  'Jaccard_score_micro': 0.19,
  'f1_micro_score': 0.31,
  'Recall_micro_score': 0.31},
 'knn': {'Accuracy': 0.27,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.28,
  'f1_micro_score': 0.44,
  'Recall_micro_score': 0.44},
 'xgboost': {'Accuracy': 0.22,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.27,
  'f1_micro_score': 0.42,
  'Recall_micro_score': 0.42},
 'Ada_Boost_Classifier': {'Accuracy': 0.22,
  'Hamming loss': 0.06,
  'Jaccard_score_micro': 0.27,
  'f1_micro_score': 0.43,
  'Recall_micro_score': 0.43}}

### Dictionnaire des scores :

In [ ]:
score_use = pd.DataFrame.from_dict(model_metrics_use)

print("-"*100)
print("-"*42 + f"Via le use :" + "-"*42)
print("-"*100)
display(score_use)

----------------------------------------------------------------------------------------------------
------------------------------------------Via le use :------------------------------------------
----------------------------------------------------------------------------------------------------


,reg_log,random_forest,decision_tree,knn,xgboost,Ada_Boost_Classifier
Accuracy,0.25,0.13,0.13,0.27,0.22,0.22
Hamming loss,0.05,0.06,0.10,0.06,0.06,0.06
Jaccard_score_micro,0.29,0.16,0.19,0.28,0.27,0.27
f1_micro_score,0.45,0.28,0.31,0.44,0.42,0.43
Recall_micro_score,0.45,0.28,0.31,0.44,0.42,0.43


#### Sauvegarde du résultat :

In [ ]:
score_use.to_csv("/content/drive/My Drive/Colab Notebooks/Stackoverflow/resultat_score_use.csv") # Cela permettra d'éviter d'attendre de relancer le notebook.

# Choix de l'algorithme supervisé :

## Visualisation des scores de toute les algorithmes utilisant des matrices d'embedding :